# HackTM 2023
## d-phi-enc 388 Points

```
In CTF, there are many people who mistakenly encrypt p, q in RSA.
But this time...
```

Attachments:

```python
from Crypto.Util.number import bytes_to_long, getStrongPrime

from secret import flag

assert len(flag) == 255
e = 3
p = getStrongPrime(1024, e=e)
q = getStrongPrime(1024, e=e)
n = p * q
phi = (p - 1) * (q - 1)
d = pow(e, -1, phi)
enc_d = pow(d, e, n)
enc_phi = pow(phi, e, n)
enc_flag = pow(bytes_to_long(flag), e, n)
print(f"{n = }")
print(f"{enc_d = }")
print(f"{enc_phi = }")
print(f"{enc_flag = }")

```
output.txt: 

In [1]:
n = 24476383567792760737445809443492789639532562013922247811020136923589010741644222420227206374197451638950771413340924096340837752043249937740661704552394497914758536695641625358888570907798672682231978378863166006326676708689766394246962358644899609302315269836924417613853084331305979037961661767481870702409724154783024602585993523452019004639755830872907936352210725695418551084182173371461071253191795891364697373409661909944972555863676405650352874457152520233049140800885827642997470620526948414532553390007363221770832301261733085022095468538192372251696747049088035108525038449982810535032819511871880097702167
enc_d = 23851971033205169724442925873736356542293022048328010529601922038597156073052741135967263406916098353904000351147783737673489182435902916159670398843992581022424040234578709904403027939686144718982884200573860698818686908312301218022582288691503272265090891919878763225922888973146019154932207221041956907361037238034826284737842344007626825211682868274941550017877866773242511532247005459314727939294024278155232050689062951137001487973659259356715242237299506824804517181218221923331473121877871094364766799442907255801213557820110837044140390668415470724167526835848871056818034641517677763554906855446709546993374
enc_phi = 3988439673093122433640268099760031932750589560901017694612294237734994528445711289776522094320029720250901589476622749396945875113134575148954745649956408698129211447217738399970996146231987508863215840103938468351716403487636203224224211948248426979344488189039912815110421219060901595845157989550626732212856972549465190609710288441075239289727079931558808667820980978069512061297536414547224423337930529183537834934423347408747058506318052591007082711258005394876388007279867425728777595263973387697391413008399180495885227570437439156801767814674612719688588210328293559385199717899996385433488332567823928840559
enc_flag = 24033688910716813631334059349597835978066437874275978149197947048266360284414281504254842680128144566593025304122689062491362078754654845221441355173479792783568043865858117683452266200159044180325485093879621270026569149364489793568633147270150444227384468763682612472279672856584861388549164193349969030657929104643396225271183660397476206979899360949458826408961911095994102002214251057409490674577323972717947269749817048145947578717519514253771112820567828846282185208033831611286468127988373756949337813132960947907670681901742312384117809682232325292812758263309998505244566881893895088185810009313758025764867

As for me the main goal was to somehow play with $\phi^3(n)$ and $d^3$ to get somewhat like linear combination in exponents. It is needed to find $d * \phi(n)\ (mod\ n)$


It's well known equation: $e * d = 1\ (mod\ \phi(n))$ and hence $e * d = 1 + K * \phi(n)$ for some $K$ which is less than $e$. Indeed: $K * \phi(n) = e * d - 1 < e * d < e * \phi(n) => K < e$. 

so.... let's just cube it

$(e  d - K  \phi(n))^3 = e^3  d^3 - 3  e^2  d^2  K  \phi(n) + 3 K^2  \phi^2(n)  e  d - K^3  \phi^3(n) = 1$

$3  e  d  K  \phi(n) *(ed - K\phi(n)) = e^3d^3 - K^3 \phi^3(n) - 1$

$3  e  d  K  \phi(n) = e^3d^3 - K^3 \phi^3(n) - 1$

Hence      
$d \phi(n) = (3 e K)^{-1}(e^3d^3 - K^3 \phi^3(n) - 1)\ (mod\ n)$

2
And for the second part we should only raise everything to the fourth power...

$(ed - K\phi(n))^4 = e^4 d^4 - 4 e^3d^3K\phi(n) + 6 e^2d^2K^2\phi^2(n) - 4 e d K^3 \phi^3(n) + K^4 \phi^4(n) = 1$

We now have the third powers of both $d$ and $\phi(n)$ and also $d \phi(n)$. So we just plug it in the equation above and with some help of extra variables we got

$A = e (e^3  d^3 - 4 K^3 \phi^3(n))\ mod(\ n)$

$B = K^4 \phi^3 - 4Kd^3e^3)\ mod(\ n)$

$C = ((6k^2(d \phi(n))^2 e^2) - 1 \ mod(\ n)$

$A * d + B * \phi(n) + C = 0\ (mod\ n)$

Surprisingly we have the second linear equation in $d$ and $\phi(n)$, and fortunately it's linearly independent of the first one, which is $e * d - K * \phi(n) = 1$

The rest is obvious, just solve the system of linear equations. Both $d$ and $\phi(n)$ are less then $n$, so we have them recovered.

In [3]:
from Crypto.Util.number import long_to_bytes
e = 3

ed = enc_d
ep = enc_phi

for k in range(1, 3):
    dphi = (pow(3 * e * k, -1, n) * (e **3 * ed - k**3 * ep -1)) % n
    
    A = (e * (e**3 * ed - 4 * k**3 * ep)) % n
    B = (k**4 * ep - k * 4 * ed * e**3) % n
    C = ((6 * k**2 * dphi**2 * e**2) - 1) % n

    m = Matrix(Zmod(n), [[A, B], [e, -k]])
    
    ans = m.solve_right(vector(Zmod(n), [-C, 1]))
    d = int(ans[0])
    phi = int(ans[1])

    a = randint(0, n)
    if (pow(a, phi, n) == 1):
        ans = pow(enc_flag, d, n)
        print(long_to_bytes(int(ans)))


b"HackTM{Have you warmed up? If not, I suggest you consider the case where e=65537, although I don't know if it's solvable. Why did I say that? Because I have to make this flag much longer to avoid solving it just by calculating the cubic root of enc_flag.}"
